<a href="https://colab.research.google.com/github/vu-topics-in-big-data-2022/examples/blob/main/example-kafka/Kafka.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Read https://towardsdatascience.com/kafka-python-explained-in-10-lines-of-code-800e3e07dad1

In [12]:
#zookeeper is packaged in a debian package that you can install
#kazoo is the python library used to work with zookeeper
!apt-get update
!apt-get install zookeeperd
!pip install kazoo

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:4 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [76.0 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [2]:
#install Java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

In [4]:
#get kafka
!wget  https://archive.apache.org/dist/kafka/2.7.0/kafka_2.13-2.7.0.tgz
!tar -zxf kafka_2.13-2.7.0.tgz
!sudo mv kafka_2.13-2.7.0 /usr/local/kafka
!sudo mkdir -p /tmp/kafka-logs

--2022-01-19 02:16:11--  https://archive.apache.org/dist/kafka/2.7.0/kafka_2.13-2.7.0.tgz
Resolving archive.apache.org (archive.apache.org)... 138.201.131.134, 2a01:4f8:172:2ec5::2
Connecting to archive.apache.org (archive.apache.org)|138.201.131.134|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68583422 (65M) [application/x-gzip]
Saving to: ‘kafka_2.13-2.7.0.tgz’

kafka_2.13-2.7.0.tg 100%[===================>]  65.41M  21.3MB/s    in 3.1s    

2022-01-19 02:16:14 (21.3 MB/s) - ‘kafka_2.13-2.7.0.tgz’ saved [68583422/68583422]



In [14]:
# We start the zookeeper service
!service zookeeper start

In [15]:
# Start the kafka server
!/usr/local/kafka/bin/kafka-server-start.sh -daemon /usr/local/kafka/config/server.properties

In [16]:
# Create a new "topic"
!/usr/local/kafka/bin/kafka-topics.sh --create --zookeeper localhost:2181 --replication-factor 1 --partitions 1 --topic bigdata

Created topic bigdata.


In [17]:
#verify the topics
!/usr/local/kafka/bin/kafka-topics.sh --zookeeper localhost:2181  --describe --topic bigdata

Topic: bigdata	PartitionCount: 1	ReplicationFactor: 1	Configs: 
	Topic: bigdata	Partition: 0	Leader: 0	Replicas: 0	Isr: 0


In [20]:
#produce a message 
# Type in messages and press enter.
# When done, manually stop the cell
!/usr/local/kafka/bin/kafka-console-producer.sh --broker-list localhost:9092 --topic bigdata

>abcd
>efg
>

In [21]:
#consume messages
# manually stop to continue, otherwise the consumer will keep on waiting for new
# messages.
!/usr/local/kafka/bin/kafka-console-consumer.sh --bootstrap-server localhost:9092 --topic bigdata --from-beginning

abcd
efg
Processed a total of 2 messages


In [24]:
#let us now try to write a simple python program to produce and consume messages.
!pip3 install kafka-python
# Create a new "topic" for python script purposes
!/usr/local/kafka/bin/kafka-topics.sh --create --zookeeper localhost:2181 --replication-factor 1 --partitions 1 --topic pybigdata

Created topic pybigdata.


In [25]:
#kafka Producer
from time import sleep
from json import dumps
from kafka import KafkaProducer
producer = KafkaProducer(bootstrap_servers=['localhost:9092'],
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))
for e in range(10):
    data = {'number' : e}
    producer.send('pybigdata', value=data)
    print("sent {}".format(data))
    sleep(2)


sent {'number': 0}
sent {'number': 1}
sent {'number': 2}
sent {'number': 3}
sent {'number': 4}
sent {'number': 5}
sent {'number': 6}
sent {'number': 7}
sent {'number': 8}
sent {'number': 9}


In [26]:
#so here is a basic consumer as well
from kafka import KafkaConsumer
from json import loads
consumer = KafkaConsumer(
    'pybigdata',
     bootstrap_servers=['localhost:9092'],
     consumer_timeout_ms=1000,#important for stopping if there is no data
     auto_offset_reset='earliest',
     value_deserializer=lambda x: loads(x.decode('utf-8')))
for message in consumer:
    message = message.value   
    print('got {}'.format(message))

got {'number': 0}
got {'number': 1}
got {'number': 2}
got {'number': 3}
got {'number': 4}
got {'number': 5}
got {'number': 6}
got {'number': 7}
got {'number': 8}
got {'number': 9}
